# Jupyter notebook based on ImageD11 to process 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 27/02/2024

Now we have good experimental parameters, we can index more grains!

In [ ]:
# There is a bug with the current version of ImageD11 in the site-wide Jupyter env.
# This has been fixed here: https://github.com/FABLE-3DXRD/ImageD11/commit/4af88b886b1775585e868f2339a0eb975401468f
# Until a new release has been made and added to the env, we need to get the latest version of ImageD11 from GitHub
# Put it in your home directory
# USER: Change the path below to point to your local copy of ImageD11:

import os

username = os.environ.get("USER")

id11_code_path = f"/home/esrf/{username}/Code/ImageD11"

import sys

sys.path.insert(0, id11_code_path)

In [ ]:
# import functions we need

import os, glob, pprint
import numpy as np
import h5py
from tqdm.notebook import tqdm

import matplotlib
%matplotlib widget
from matplotlib import pyplot as plt

# import utils
from ImageD11.nbGui import nb_utils as utils

import ImageD11.grain
import ImageD11.indexing
import ImageD11.columnfile
from ImageD11.sinograms import properties, dataset

from ImageD11.blobcorrector import eiger_spatial

In [ ]:
cmp = {'compression':'gzip',
       'compression_opts': 2,
       'shuffle' : True }

def save_array(grp, name, ary):
    hds = grp.require_dataset(name, 
                              shape=ary.shape,
                              dtype=ary.dtype,
                              **cmp)
    hds[:] = ary
    return hds

def save_grains(grains, ds):
    with h5py.File(ds.grainsfile, 'w') as hout:
        grn = hout.create_group('grains')
        for g in tqdm(grains):
            gg = grn.create_group(str(g.gid))
            save_array(gg, 'peaks_3d_indexing', g.peaks_3d).attrs['description'] = "Strong 3D peaks that were assigned to this grain during indexing"
            gg.attrs.update({'ubi':g.ubi,
                            'translation':g.translation})

In [ ]:
### USER: specify your experimental directory

rawdata_path = "/home/esrf/james1997a/Data/ihma439/id11/20231211/RAW_DATA"

!ls -lrt {rawdata_path}

processed_data_root_dir = "/home/esrf/james1997a/Data/ihma439/id11/20231211/PROCESSED_DATA/James/20240226"  # USER: modify this to change the destination folder if desired

In [ ]:
# USER: pick a sample and a dataset you want to segment

sample = "FeAu_0p5_tR"
dataset = "ff1"

In [ ]:
# desination of H5 files

dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")

# USER: specify the path to the parameter file

parfile = '/home/esrf/james1997a/Data/ihma439/id11/20231211/PROCESSED_DATA/James/20240226/Fe_tdxrd_refined.par'

In [ ]:
# load the dataset from file

ds = ImageD11.sinograms.dataset.load(dset_path)

print(ds)
print(ds.shape)

In [ ]:
# load 3d columnfile from disk

cf_3d = ImageD11.columnfile.colfile_from_hdf(ds.col3dfile)

cf_3d.parameters.loadparameters(parfile)
cf_3d.updateGeometry()

if "index" not in cf_3d.titles:
    cf_3d.addcolumn(np.arange(cf_3d.nrows), "index")

In [ ]:
# plot the 3D peaks (fewer of them) as a cake (two-theta vs eta)
# if the parameters in the par file are good, these should look like straight lines

fig, ax = plt.subplots()

ax.scatter(cf_3d.ds, cf_3d.eta, s=1)

ax.set_xlabel("D-star")
ax.set_ylabel("eta")

plt.show()

In [ ]:
# here we are filtering our peaks (cf_2d) to select only the strongest ones for indexing purposes only!
# dsmax is being set to limit rings given to the indexer
# because we have so many 3d peaks, we will filter down to 3 rings

# USER: modify the "frac" parameter below and re-run the cell until we have around 25,000 peaks for indexing

cf_strong = utils.selectpeaks(cf_3d, frac=0.95, dsmax=0.92, doplot=0.05, dstol=0.01)
print(f"Got {cf_strong.nrows} strong peaks for indexing")
cf_strong.writefile(f'{sample}_{dataset}_3d_peaks_strong.flt')

In [ ]:
# we will also export some additional strong peaks across all rings
# this will be useful for grain refinement later (using makemap)

cf_strong_allrings = utils.selectpeaks(cf_3d, frac=0.95, dsmax=cf_3d.ds.max(), doplot=0.8, dstol=0.01)
print(f"Got {cf_strong_allrings.nrows} strong peaks for makemap")
cf_strong_allrings_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt'
cf_strong_allrings.writefile(cf_strong_allrings_path)

In [ ]:
# now we can take a look at the intensities of the remaining peaks

fig, ax = plt.subplots()

ax.plot(cf_strong.ds, cf_strong.sum_intensity,',')
ax.semilogy()

ax.set_xlabel("D-star")
ax.set_ylabel("Intensity")

plt.show()

In [ ]:
# now we can define a unit cell from our parameters

Fe = ImageD11.unitcell.unitcell_from_parameters(cf_strong.parameters)
Fe.makerings(cf_strong.ds.max())

In [ ]:
# now let's plot our peaks again, with the rings from the unitcell included, to check our lattice parameters are good

fig, ax = plt.subplots()

skip=1
ax.scatter( cf_strong.ds[::skip], cf_strong.eta[::skip], s=0.5)
ax.plot( Fe.ringds, [0,]*len(Fe.ringds), '|', ms=90, c='orange')
ax.set_xlabel('1 / d ($\AA$)')
ax.set_ylabel('$\\eta$ (deg)')

plt.show()

In [ ]:
# specify our ImageD11 indexer with these peaks

indexer = ImageD11.indexing.indexer_from_colfile(cf_strong)

print(f"Indexing {cf_strong.nrows} peaks")

# USER: set a tolerance in d-space (for assigning peaks to powder rings)

indexer.ds_tol = 0.05

# change the log level so we can see what the ring assigments look like

ImageD11.indexing.loglevel = 1

# assign peaks to powder rings

indexer.assigntorings()

# change log level back again

ImageD11.indexing.loglevel = 3

In [ ]:
# now we are indexing!
# indexing will select all rings with a multiplicity below max_multiplity to search
max_multiplicity = 13
# the minimum number of peaks on a ring for a ring to be indexed on
min_counts_on_ring = 0
# the sequence of hkl tolerances the indexer will iterate through
hkl_tols_seq = [0.01, 0.02, 0.03, 0.04, 0.05, 0.1]
# the sequence of minpks fractions the indexer will iterate through
fracs = [0.9, 0.75]
# the tolerance in g-vector angle
cosine_tol = np.cos(np.radians(90.25))
# the max number of UBIs we can find per pair of rings
max_grains = 1000

grains, indexer = utils.do_index(cf=cf_strong,
                                dstol=indexer.ds_tol,
                                max_mult=max_multiplicity,
                                min_ring_count=min_counts_on_ring,
                                hkl_tols=hkl_tols_seq,
                                fracs=fracs,
                                cosine_tol=cosine_tol,
                                max_grains=max_grains
)

In [ ]:
# create grain objects
grains = [ImageD11.grain.grain(ubi, translation=np.array([0., 0., 0.])) for ubi in indexer.ubis]

# set grain GIDs (useful if we ever delete a grain)
for i, g in enumerate(grains):
    g.gid = i

In [ ]:
# plot pole figures?

In [ ]:
tmp_ubi_path = f'{sample}_{dataset}_grains.ubi'
tmp_map_path = f'{sample}_{dataset}_grains.map'

new_flt_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt.new'  # flt file containing assignments from makemap
unindexed_flt_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt.unindexed'  # remaining unassigned peaks from makemap

In [ ]:
ImageD11.grain.write_grain_file(tmp_ubi_path, grains)

In [ ]:
omegas_sorted = np.sort(ds.omega)[0]
omega_slop = np.round(np.diff(omegas_sorted).mean(), 3)

makemap_hkl_tol_seq = [0.05, 0.025, 0.01]

In [ ]:
for inc, makemap_tol in enumerate(makemap_hkl_tol_seq):
    print(f"Running makemap {inc+1}/{len(makemap_hkl_tol_seq)}")
    if inc == 0:  # ubi into map
        makemap_output = !makemap.py -p {parfile} -u {tmp_ubi_path} -U {tmp_map_path} -f {cf_strong_allrings_path} -F {unindexed_flt_path} -s cubic -t {makemap_hkl_tol_seq[inc]} --omega_slop={omega_slop} --no_sort
    else:  # map into map
        makemap_output = !makemap.py -p {parfile} -u {tmp_map_path} -U {tmp_map_path} -f {cf_strong_allrings_path} -F {unindexed_flt_path} -s cubic -t {makemap_hkl_tol_seq[inc]} --omega_slop={omega_slop} --no_sort

In [ ]:
# re-import our refined grains from the makemap procedure

grains2 = ImageD11.grain.read_grain_file(tmp_map_path)

In [ ]:
# remove grains with no peaks

grains2 = [grain for grain in grains2 if "no peaks" not in grain.intensity_info]

In [ ]:
centre_plot = False

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
xx = [grain.translation[0] for grain in grains2]
yy = [grain.translation[1] for grain in grains2]
zz = [grain.translation[2] for grain in grains2]
# col = [utils.grain_to_rgb(grain) for grain in grains2]  # IPF-Z colour instead
col = [float(grain.npks) for grain in grains2]
sizes = [0.01*(float(grain.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", ""))) for grain in grains2]
if centre_plot:
    scatterplot = ax.scatter(xx-np.mean(xx), yy-np.mean(yy), zz, c=col, s=sizes)
else:
    scatterplot = ax.scatter(xx, yy, zz, c=col, s=sizes)
ax.set_xlim(-200,200)
ax.set_ylim(-200,200)
ax.set_zlim(-200,200)
plt.colorbar(scatterplot)
ax.set_title("Grains coloured by n peaks")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist([float(grain.npks) for grain in grains2], bins=30)
plt.show()

In [ ]:
# find the spike
absolute_minpks = 25

In [ ]:
# filter out grains with fewer than 15 peaks
grains_filtered = [grain for grain in grains2 if float(grain.npks) > absolute_minpks]

In [ ]:
centre_plot = False

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
xx = [grain.translation[0] for grain in grains_filtered]
yy = [grain.translation[1] for grain in grains_filtered]
zz = [grain.translation[2] for grain in grains_filtered]
# col = [utils.grain_to_rgb(grain) for grain in grains_filtered]  # IPF-Z colour instead
col = [float(grain.npks) for grain in grains_filtered]
sizes = [0.01*(float(grain.intensity_info.split("mean = ")[1].split(" , ")[0].replace("'", ""))) for grain in grains_filtered]
if centre_plot:
    scatterplot = ax.scatter(xx-np.mean(xx), yy-np.mean(yy), zz, c=col, s=sizes)
else:
    scatterplot = ax.scatter(xx, yy, zz, c=col, s=sizes)
ax.set_xlim(-200,200)
ax.set_ylim(-200,200)
ax.set_zlim(-200,200)
plt.colorbar(scatterplot)
ax.set_title("Grains coloured by n peaks")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

In [ ]:
for grain in grains_filtered:
    grain.gid = int(grain.name.split(":")[0])

In [ ]:
# now we will assign our strong peaks across all rings to the grains
# so we can save the data to file for later

In [ ]:
tol = 0.05

utils.assign_peaks_to_grains(grains_filtered, cf_strong_allrings, tol)

print("Storing peak data in grains")
# iterate through all the grains
for g in tqdm(grains_filtered):
    # store this grain's peak indices so we know which 4D peaks we used for indexing
    g.peaks_3d = cf_strong_allrings.index[cf_strong_allrings.grain_id == g.gid]

In [ ]:
fig, ax = plt.subplots()
ax.hist([np.mean(grain.unitcell[0:3]) for grain in grains_filtered], bins=25)
plt.show()

print(np.mean([np.mean(grain.unitcell[0:3]) for grain in grains_filtered]))

In [ ]:
# save grain data

save_grains(grains_filtered, ds)

In [ ]:
# Now that we're happy with our indexing parameters, we can define our indexing function to run for the rest of our data
# just modify the parameters at the start of the cell below according to what you found worked well above
# then run the two cells below

In [ ]:
# Now that we're happy with our segmentation parameters, we can run the below cell to do this in bulk for many samples/datasets
# by default this will do all samples in sample_list, all datasets with a prefix of dset_prefix
# you can add samples and datasets to skip

skips_dict = {
    "FeAu_0p5_tR": []
}

dset_prefix = "ff"

sample_list = ["FeAu_0p5_tR"]
    
samples_dict = utils.find_datasets_to_process(rawdata_path, skips_dict, dset_prefix, sample_list)

parfile = '/home/esrf/james1997a/Data/ihma439/id11/20231211/PROCESSED_DATA/James/20240226/Fe_tdxrd_refined.par'

cf_strong_frac = 0.95
cf_strong_dsmax = 0.92
cf_strong_dstol = 0.01

cf_strong_allrings_frac = 0.95
cf_strong_allrings_dstol = 0.01

indexer_dstol = 0.05
indexer_max_mult = 13
indexer_hkl_tols = [0.01, 0.02, 0.03, 0.04, 0.05, 0.1]
indexer_fracs = [0.9, 0.75]
indexer_cosine_tol = np.cos(np.radians(90.25))
indexer_max_grains = 1000
indexer_min_ring_count = 0

makemap_hkl_tol_seq = [0.05, 0.025, 0.01]
makemap_import_minpks = 25

peak_assignment_hkl_tol = 0.05

for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        print("Importing DataSet object")
        dset_path = os.path.join(processed_data_root_dir, sample, f"{sample}_{dataset}", f"{sample}_{dataset}_dataset.h5")
        ds = ImageD11.sinograms.dataset.load(dset_path)
        print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
        
        if os.path.exists(ds.grainsfile):
            print(f"Found existing grains file for {dataset} in {sample}, skipping")
            continue


        print("Loading 3D peaks")
        cf_3d = ImageD11.columnfile.colfile_from_hdf(ds.col3dfile)
        cf_3d.parameters.loadparameters(parfile)
        cf_3d.updateGeometry()
        if "index" not in cf_3d.titles:
            cf_3d.addcolumn(np.arange(cf_3d.nrows), "index")

        print("Filtering 3D peaks")
        cf_strong = utils.selectpeaks(cf_3d, frac=cf_strong_frac, dsmax=cf_strong_dsmax, doplot=None, dstol=cf_strong_dstol)
        print(f"Got {cf_strong.nrows} strong peaks for indexing")
        cf_strong_path = f'{sample}_{dataset}_3d_peaks_strong.flt'
        cf_strong.writefile(cf_strong_path)

        cf_strong_allrings = utils.selectpeaks(cf_3d, frac=cf_strong_allrings_frac, dsmax=cf_3d.ds.max(), doplot=None, dstol=cf_strong_allrings_dstol)
        print(f"Got {cf_strong_allrings.nrows} strong peaks for makemap")
        cf_strong_allrings_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt'
        cf_strong_allrings.writefile(cf_strong_allrings_path)
        
        grains, indexer = utils.do_index(cf=cf_strong,
                                dstol=indexer_dstol,
                                max_mult=indexer_max_mult,
                                min_ring_count=indexer_min_ring_count,
                                hkl_tols=indexer_hkl_tols,
                                fracs=indexer_fracs,
                                cosine_tol=indexer_cosine_tol,
                                max_grains=indexer_max_grains
                                )

        grains = [ImageD11.grain.grain(ubi, translation=np.array([0., 0., 0.])) for ubi in indexer.ubis]

        for i, g in enumerate(grains):
            g.gid = i

        tmp_ubi_path = f'{sample}_{dataset}_grains.ubi'
        tmp_map_path = f'{sample}_{dataset}_grains.map'

        new_flt_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt.new'  # flt file containing assignments from makemap
        unindexed_flt_path = f'{sample}_{dataset}_3d_peaks_strong_all_rings.flt.unindexed'  # remaining unassigned peaks from makemap

        ImageD11.grain.write_grain_file(tmp_ubi_path, grains)

        omegas_sorted = np.sort(ds.omega)[0]
        omega_slop = np.round(np.diff(omegas_sorted).mean(), 3)

        makemap_hkl_tol_seq = makemap_hkl_tol_seq

        for inc, makemap_tol in enumerate(makemap_hkl_tol_seq):
            print(f"Running makemap {inc+1}/{len(makemap_hkl_tol_seq)}")
            if inc == 0:  # ubi into map
                makemap_output = !makemap.py -p {parfile} -u {tmp_ubi_path} -U {tmp_map_path} -f {cf_strong_allrings_path} -F {unindexed_flt_path} -s cubic -t {makemap_hkl_tol_seq[inc]} --omega_slop={omega_slop} --no_sort
            else:  # map into map
                makemap_output = !makemap.py -p {parfile} -u {tmp_map_path} -U {tmp_map_path} -f {cf_strong_allrings_path} -F {unindexed_flt_path} -s cubic -t {makemap_hkl_tol_seq[inc]} --omega_slop={omega_slop} --no_sort

        grains2 = ImageD11.grain.read_grain_file(tmp_map_path)
        
        # remove grains with no peaks
        grains2 = [grain for grain in grains2 if "no peaks" not in grain.intensity_info]
        
        absolute_minpks = makemap_import_minpks
        grains_filtered = [grain for grain in grains2 if float(grain.npks) > absolute_minpks]

        for grain in grains_filtered:
            grain.gid = int(grain.name.split(":")[0])

        tol = peak_assignment_hkl_tol

        utils.assign_peaks_to_grains(grains_filtered, cf_strong_allrings, tol)

        print("Storing peak data in grains")
        # iterate through all the grains
        for g in tqdm(grains_filtered):
            # store this grain's peak indices so we know which 4D peaks we used for indexing
            g.peaks_3d = cf_strong_allrings.index[cf_strong_allrings.grain_id == g.gid]

        print("Saving grains")
        save_grains(grains_filtered, ds)

        if os.path.exists(cf_strong_path):
            os.remove(cf_strong_path)

        if os.path.exists(cf_strong_allrings_path):
            os.remove(cf_strong_allrings_path)

        if os.path.exists(tmp_ubi_path):
            os.remove(tmp_ubi_path)

        if os.path.exists(tmp_map_path):
            os.remove(tmp_map_path)

        if os.path.exists(new_flt_path):
            os.remove(new_flt_path)

        if os.path.exists(unindexed_flt_path):
            os.remove(unindexed_flt_path)